## Start

In [1]:
import json
import os
import torch
import tqdm
from shutil import copyfile

# Detectron imports
from detectron2.engine import launch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.data import build_detection_test_loader, MetadataCatalog

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from train_utils import ActiveTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml'
args.random_seed = 1000
args.resume=True
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50162', eval_only=False, inference_config='/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=1000, resume=True, test_dataset='bdd_val')


In [3]:
# run this once per session only
cfg = setup_config(args, random_seed=args.random_seed, is_testing=False)

Loading config /home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.


[02/05 17:30:10 detectron2]: Rank of current process: 0. World size: 1
[02/05 17:30:10 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.7.1 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0                   Tesla V100-SXM2-32GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow                  8.1.0
torchvision             0.8.2 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torchvision
torchvision arch flags  3.5, 5.0

[02/05 17:30:11 detectron2]: Full config saved to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_1000/config.yaml


In [8]:
#cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

# Make sure only 1 data point is processed at a time. This simulates
# deployment.
cfg.defrost()
cfg.DATALOADER.NUM_WORKERS = 32
cfg.SOLVER.IMS_PER_BATCH = 1

cfg.MODEL.DEVICE = device.type

# Set up number of cpu threads
torch.set_num_threads(cfg.DATALOADER.NUM_WORKERS)

# Create inference output directory and copy inference config file to keep
# track of experimental settings
inference_output_dir = os.path.join(
    cfg['OUTPUT_DIR'],
    'inference',
    args.test_dataset,
    os.path.split(args.inference_config)[-1][:-5])
os.makedirs(inference_output_dir, exist_ok=True)
copyfile(args.inference_config, os.path.join(
    inference_output_dir, os.path.split(args.inference_config)[-1]))

# Get category mapping dictionary:
train_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
test_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    args.test_dataset).thing_dataset_id_to_contiguous_id

# If both dicts are equal or if we are performing out of distribution
# detection, just flip the test dict.
if (train_thing_dataset_id_to_contiguous_id == test_thing_dataset_id_to_contiguous_id) or (
        cfg.DATASETS.TRAIN[0] == 'coco_not_in_voc_2017_train'):
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
else:
    # If not equal, two situations: 1) BDD to KITTI and 2) COCO to PASCAL
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
    if 'voc' in args.test_dataset and 'coco' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.COCO_TO_VOC_CONTIGUOUS_ID.items())
    elif 'kitti' in args.test_dataset and 'bdd' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.BDD_TO_KITTI_CONTIGUOUS_ID.items())
    else:
        ValueError(
            'Cannot generate category mapping dictionary. Please check if training and inference datasets are compatible.')
    cat_mapping_dict = dict(
        (dataset_mapping_dict[k], v) for k, v in cat_mapping_dict.items())

# Build predictor
model = build_model(cfg)

DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            "/home/richard.tanai/cvpr2/pod_compare/data_backup/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/model_final.pth", resume=False)

trainer = ActiveTrainer(cfg, model)
#trainer.resume_or_load(resume=True)

#predictor = build_predictor(cfg, model)



[02/05 17:32:53 fvcore.common.checkpoint]: Loading checkpoint from /home/richard.tanai/cvpr2/pod_compare/data_backup/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/model_final.pth
[02/05 17:32:59 d2.data.datasets.coco]: Loading /public-dataset/BDD/bdd100k/labels/train_coco_format.json takes 5.27 seconds.
[02/05 17:32:59 d2.data.datasets.coco]: Loaded 69863 images in COCO format from /public-dataset/BDD/bdd100k/labels/train_coco_format.json
[02/05 17:33:02 d2.data.build]: Removed 458 images with no usable annotations. 69405 images left.
[02/05 17:33:06 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(720,), max_size=1333, sample_style='choice'), RandomFlip()]


In [9]:
predictor = build_predictor(cfg, model)

test_data_loader = build_detection_test_loader(
    cfg, dataset_name=args.test_dataset)

  0%|          | 11/10000 [00:06<1:42:02,  1.63it/s]


In [ ]:
# 10000 is already started in the init
# epoch is 20, just an arbitrary number lol
train_step = 1
label_per_step = 100
while(1):
    print(f"performing train step {train_step}")
    trainer.train()
    torch.save(model.state_dict(), f"outputs/checkpoint_step{train_step}.pth")
    if len(trainer.dataset) >= label_per_step:
        trainer.dataset.label_randomly(label_per_step)
    elif len(trainer.dataset) > 0:
        trainer.datasaet.label_randomly(len(trainer.dataset))
    else:
        break
    trainer.rebuild_trainer()
    train_step += 1

performing train step 1
[02/03 19:54:32 d2.engine.train_loop]: Starting training from iteration 0
[02/03 19:54:47 d2.utils.events]:  eta: 0:00:39  iter: 99  total_loss: 0.1649  loss_cls: 0.04713  loss_box_reg: 0.1227  time: 0.1348  data_time: 0.0030  lr: 0.00024975  max_mem: 2432M
[02/03 19:55:01 d2.utils.events]:  eta: 0:00:26  iter: 199  total_loss: 0.2076  loss_cls: 0.05903  loss_box_reg: 0.1383  time: 0.1342  data_time: 0.0029  lr: 0.0004995  max_mem: 2432M
[02/03 19:55:14 d2.utils.events]:  eta: 0:00:13  iter: 299  total_loss: 0.2091  loss_cls: 0.05158  loss_box_reg: 0.1496  time: 0.1338  data_time: 0.0029  lr: 0.00074925  max_mem: 2432M
[02/03 19:55:28 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_333/model_final.pth
[02/03 19:55:28 d2.utils.events]:  eta: 0:00:00  iter: 399  total_loss: 0.2143  loss_cls: 0.05284  loss_box_reg: 0.1314  time: 0.1339  data_time

[02/03 20:02:42 d2.utils.events]:  eta: 0:00:53  iter: 1199  total_loss: 0.1498  loss_cls: 0.02695  loss_box_reg: 0.1131  time: 0.1342  data_time: 0.0026  lr: 0.0025  max_mem: 2432M
[02/03 20:02:56 d2.utils.events]:  eta: 0:00:40  iter: 1299  total_loss: 0.158  loss_cls: 0.01868  loss_box_reg: 0.1294  time: 0.1342  data_time: 0.0028  lr: 0.0025  max_mem: 2432M
[02/03 20:03:09 d2.utils.events]:  eta: 0:00:27  iter: 1399  total_loss: 0.165  loss_cls: 0.02679  loss_box_reg: 0.1155  time: 0.1343  data_time: 0.0031  lr: 0.0025  max_mem: 2432M
[02/03 20:03:23 d2.utils.events]:  eta: 0:00:13  iter: 1499  total_loss: 0.1426  loss_cls: 0.01527  loss_box_reg: 0.1248  time: 0.1343  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:03:36 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_333/model_final.pth
[02/03 20:03:37 d2.utils.events]:  eta: 0:00:00  iter: 1599  total_lo

[02/03 20:11:17 d2.utils.events]:  eta: 0:02:14  iter: 1399  total_loss: 0.1254  loss_cls: 0.01125  loss_box_reg: 0.1139  time: 0.1344  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:11:30 d2.utils.events]:  eta: 0:02:01  iter: 1499  total_loss: 0.1187  loss_cls: 0.00807  loss_box_reg: 0.09897  time: 0.1344  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:11:44 d2.utils.events]:  eta: 0:01:47  iter: 1599  total_loss: 0.09336  loss_cls: 0.008918  loss_box_reg: 0.0817  time: 0.1343  data_time: 0.0031  lr: 0.0025  max_mem: 2432M
[02/03 20:11:57 d2.utils.events]:  eta: 0:01:34  iter: 1699  total_loss: 0.1094  loss_cls: 0.007791  loss_box_reg: 0.09254  time: 0.1342  data_time: 0.0028  lr: 0.0025  max_mem: 2432M
[02/03 20:12:11 d2.utils.events]:  eta: 0:01:20  iter: 1799  total_loss: 0.09628  loss_cls: 0.01202  loss_box_reg: 0.07901  time: 0.1343  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:12:24 d2.utils.events]:  eta: 0:01:07  iter: 1899  total_loss: 0.101  los

[02/03 20:19:51 d2.engine.hooks]: Overall training speed: 2798 iterations in 0:06:16 (0.1346 s / it)
[02/03 20:19:51 d2.engine.hooks]: Total training time: 0:06:17 (0:00:01 on hooks)
performing train step 8
[02/03 20:19:51 d2.engine.train_loop]: Starting training from iteration 0
[02/03 20:20:04 d2.utils.events]:  eta: 0:06:56  iter: 99  total_loss: 0.0966  loss_cls: 0.00805  loss_box_reg: 0.08493  time: 0.1342  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:20:18 d2.utils.events]:  eta: 0:06:42  iter: 199  total_loss: 0.08495  loss_cls: 0.003859  loss_box_reg: 0.07971  time: 0.1341  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:20:31 d2.utils.events]:  eta: 0:06:29  iter: 299  total_loss: 0.09162  loss_cls: 0.00396  loss_box_reg: 0.08664  time: 0.1340  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:20:45 d2.utils.events]:  eta: 0:06:16  iter: 399  total_loss: 0.06917  loss_cls: 0.003399  loss_box_reg: 0.06557  time: 0.1340  data_time: 0.0030  lr: 0.0025  ma

In [ ]:
len(trainer.dataset.pool)

In [ ]:
# coco has 80000 images
# the max iter is 90000 and batchsize of 4, making it about 4.5epochs
# for this study 4 epochs will be used instead
# no weights refresh will be used
# hooks are now rebuilt

In [4]:
## prediction outputs
# in what order are the outputs arranged, they are arranged by the max clas pred score


# to do
# change test dataloader to pool dataloader
# use 

In [115]:
final_output_list = []
cls_score_list = []
box_score_list = []


if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)
                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                results = outputs
                
                cls_preds = results.pred_cls_probs.cpu().numpy()
                predicted_boxes = results.pred_boxes.tensor.cpu().numpy()
                predicted_covar_mats = results.pred_boxes_covariance.cpu().numpy()
                
                box_score = np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()
                #mean of the max confidence
                cls_score = cls_preds.max(axis=1).mean()
                #change cls_score to entropy next time
                box_score_list.append(box_score)
                cls_score_list.append(cls_score)
                
                pbar.update(1)
                
                
cls_score_rank = np.array(cls_score_list).argsort().argsort()
box_score_rank = (-np.array(box_score_list)).argsort().argsort()
total_sort = np.argsort(cls_score_rank + box_score_rank)
idx_to_label = total_sort[:10000].tolist()

  0%|          | 11/10000 [00:06<1:39:44,  1.67it/s]


In [ ]:
#sort from most uncertain to least uncertain

In [134]:
cls_score_rank = np.array(cls_score_list).argsort().argsort()
box_score_rank = (-np.array(box_score_list)).argsort().argsort()
total_sort = np.argsort(cls_score_rank + box_score_rank)
idx_to_label = total_sort[:3].tolist()

In [135]:
idx_to_label

[5, 10, 1]

In [122]:
cls_score_list

[0.27794713,
 0.13429637,
 0.16547583,
 0.24252155,
 0.25121427,
 0.10372658,
 0.21627478,
 0.23667419,
 0.15529655,
 0.21689896,
 0.20493267]

In [121]:
cls_sort.argsort()

array([10,  1,  3,  8,  9,  0,  5,  7,  2,  6,  4])

In [123]:
cls_sort.argsort().argsort()

array([ 5,  1,  8,  2, 10,  6,  9,  7,  3,  4,  0])

In [124]:
cls_sort.argsort().argsort().argsort()

array([10,  1,  3,  8,  9,  0,  5,  7,  2,  6,  4])

In [114]:
box_score_list

[757717600000.0,
 188202800000.0,
 137757930000.0,
 177554420000.0,
 223814990000000.0,
 28815011000000.0,
 121507410000.0,
 5206625500000.0,
 54316253000.0,
 25104515000.0,
 79928750000000.0]

In [107]:
(-cls_score_list*np.log(cls_score_list)).sum()

3.4716692

In [125]:
cls_score_rank = np.array(cls_score_list).argsort().argsort()

In [126]:
box_score_rank = (-np.array(box_score_list)).argsort().argsort()

In [130]:
total_sort = np.argsort(cls_score_rank + box_score_rank)

In [131]:
print(cls_score_list)
print(box_score_list)
print(cls_score_rank + box_score_rank)

[0.27794713, 0.13429637, 0.16547583, 0.24252155, 0.25121427, 0.10372658, 0.21627478, 0.23667419, 0.15529655, 0.21689896, 0.20493267]
[1759573600000.0, 174372500000.0, 1790520200.0, 563659400000.0, 300968420000000.0, 25916747000000.0, 71347390000.0, 6429390400000.0, 61955970000.0, 17331331000.0, 53910260000000.0]
[14  7 13 13  9  2 12 10 10 15  5]


In [103]:
step=3

In [104]:
total_sort[:step]

array([10,  5,  0])

In [99]:
cls_sort

array([ 5,  1,  8,  2, 10,  6,  9,  3,  7,  4,  0])

In [98]:
box_sort

array([ 4, 10,  5,  7,  0,  1,  3,  8,  6,  9,  2])

In [85]:
cls_score_list = np.array(cls_score_list)

In [89]:
np.argsort(-cls_score_list)

array([ 0,  4,  7,  3,  9,  6, 10,  2,  8,  1,  5])

In [87]:
np.argsort(norm)

array([ 2,  9,  6,  8,  3,  1,  0,  7,  5, 10,  4])

In [90]:
np.argsort(np.argsort(-cls_score_list)+np.argsort(norm))

array([ 0,  5,  7, 10,  6,  3,  9,  4,  1,  2,  8])

In [54]:
results = outputs

cls_preds = results.pred_cls_probs.cpu().numpy()
predicted_boxes = results.pred_boxes.tensor.cpu().numpy()
predicted_covar_mats = results.pred_boxes_covariance.cpu().numpy()

box_score = np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()
cls_score = cls_preds.max(axis=1).mean()

In [26]:
predicted_boxes[0].shape

(4,)

In [55]:
cls_preds

array([[0.9514904 , 0.01849801, 0.05607532, 0.05667713, 0.0051091 ,
        0.00721828, 0.01677718],
       [0.9232524 , 0.03924947, 0.06935471, 0.05374594, 0.01577606,
        0.02107754, 0.03311143],
       [0.92191714, 0.0115991 , 0.02670465, 0.04891825, 0.00488035,
        0.00895986, 0.02172219],
       [0.89119476, 0.01955939, 0.03910834, 0.03826052, 0.01360259,
        0.01586567, 0.03685952],
       [0.8830716 , 0.02215327, 0.04523037, 0.0299987 , 0.01517591,
        0.02532225, 0.06861553],
       [0.870052  , 0.02633451, 0.04346598, 0.03136223, 0.01143321,
        0.01095212, 0.02682063],
       [0.8248973 , 0.01407167, 0.04847713, 0.04400839, 0.01332223,
        0.02010958, 0.05683623],
       [0.815567  , 0.01300281, 0.04869989, 0.0770672 , 0.01309281,
        0.04011224, 0.07494085],
       [0.81320095, 0.0286712 , 0.06175374, 0.03810425, 0.01602989,
        0.01672141, 0.02791771],
       [0.8129124 , 0.05066876, 0.13602476, 0.06443601, 0.00952293,
        0.0219253 , 0.0

In [45]:
predicted_covar_mats[0]

array([[ 5.56829   , -0.01457549,  1.4243178 ,  0.02851192],
       [-0.01457549,  3.0298598 ,  0.01219342,  0.80456716],
       [ 1.4243178 ,  0.01219342,  5.6584744 ,  0.01624925],
       [ 0.02851192,  0.80456716,  0.01624925,  3.0154514 ]],
      dtype=float32)

In [27]:
predicted_covar_mats[0].shape

(4, 4)

In [24]:
predicted_covar_mats[0].diagonal().prod()

287.8698

In [66]:
cls_preds.max(axis=1).mean()

0.29015502

In [60]:
for mat in cls_preds:
    print(mat.max())

0.9514904
0.9232524
0.92191714
0.89119476
0.8830716
0.870052
0.8248973
0.815567
0.81320095
0.8129124
0.80886
0.77521163
0.69447565
0.59073365
0.50749236
0.49180537
0.4530619
0.44125104
0.4229238
0.4117765
0.40858513
0.35378405
0.34403086
0.33916226
0.30780837
0.30204326
0.29606
0.28628227
0.28322682
0.27005008
0.26971042
0.260329
0.24800633
0.24103594
0.23950827
0.2296642
0.22665642
0.22636071
0.21685438
0.20248018
0.19498096
0.19125111
0.18951294
0.18223882
0.18046384
0.17863756
0.17813516
0.17662005
0.17539997
0.1740193
0.1659687
0.16589613
0.15962568
0.15757553
0.15658474
0.15624736
0.14929207
0.14630942
0.1425082
0.14196514
0.13636878
0.1355774
0.12341988
0.09820169
0.0863961
0.083679035
0.08306127
0.075409494
0.07488375
0.074635044
0.07308651
0.07187654
0.07116739
0.07067328
0.06839184
0.06590837
0.06522693
0.06395777
0.061402667
0.059782505
0.059094127
0.057855528
0.053887915
0.053795476
0.053523432
0.051514357
0.050694115


In [62]:
def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation

In [64]:
for mat in predicted_covar_mats:
    print(mat.diagonal().prod())
    #print(correlation_from_covariance(mat))

287.8698
9.45183
929.9821
226.54271
2774.8628
111.6626
1368.9465
173.05153
255.85495
87310.38
386.8067
1105.3667
4284.7305
56834030.0
62451600000.0
3398949.8
28475788.0
58315216.0
184771790.0
110986140.0
1979667.2
20779422.0
1286523400.0
65738650.0
55346990.0
76672340.0
220722690.0
920491.56
11903610.0
175951600.0
116654830.0
2549750.5
342720420.0
860921540.0
245255540.0
187860000.0
4998993.5
679783800.0
43780508.0
457226.2
987850750.0
168627730.0
304424500.0
80069390.0
606926850.0
414725700.0
65004716.0
593403900.0
156146450.0
805560.0
7374736.5
734853000.0
141763540.0
182407860.0
180282610.0
541565400.0
5137318400.0
64569384.0
812142900.0
250753200.0
278911940.0
14129849000.0
556604100000.0
32075647000.0
8409044000000.0
156451770000.0
10702015000000.0
3749178200000.0
637937300.0
456493040000.0
100858950000000.0
56943096000.0
255043500000.0
170220650000.0
263956610000.0
685287200000.0
5251395600.0
1003056400000.0
13581854000.0
14627200000.0
23695426000.0
45550940000.0
93554790000.0
87

In [40]:
def mult_diag(x):
    print(x)
    return x.diagonal().prod()

In [74]:
np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()

1481842400000.0

In [ ]:
np.apply_over_axes